In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
from src.graph_file_iterator import GraphFileIterator
from tqdm import tqdm
from src.var import *

In [ ]:
ego_users = pd.DataFrame(
    data=set([int(os.path.splitext(f)[0]) for f in os.listdir(TWITTER_DATA_DIR)]),
    columns=['user_id']
)

ego_users_set = set(ego_users['user_id'].astype(int))

ego_users.to_pickle(USERS_DATA)

ego_users.head()

In [ ]:
related_users = pd.DataFrame(columns=['ego_user_id', 'user_id'])

for circles_filename, circles_filepath in tqdm(GraphFileIterator(TWITTER_DATA_DIR, '.circles')):
    user_id = int(os.path.splitext(circles_filename)[0])
    related = set()

    for row in csv.reader(open(circles_filepath), delimiter='\t'):
        circle_users = set(map(int, row[2:]))
        related.update(set.intersection(ego_users_set, circle_users))

    for related_user in related:
        related_users.loc[len(related_users)] = [user_id, related_user]

related_users.to_pickle(RELATED_USERS_DATA)

related_users

In [ ]:
followers = pd.DataFrame(columns=['source', 'target'])

for followers_filename, followers_filepath in tqdm(GraphFileIterator(TWITTER_DATA_DIR, '.edges')):
    source = int(os.path.splitext(followers_filename)[0])

    for row in csv.reader(open(followers_filepath), delimiter=' '):
        u1 = int(row[0])
        u2 = int(row[1])

        for target in [u1, u2]:
            if target in ego_users_set:
                followers.loc[len(followers)] = [source, target]

followers = followers.drop_duplicates()

followers.to_pickle(FOLLOWERS_DATA)

followers

In [ ]:
def load_features(node_id):
    features = []

    for row in csv.reader(open(os.path.join(TWITTER_DATA_DIR, f'{node_id}.feat'), encoding='latin1'), delimiter=' '):
        row_id = row[0]
        feature = row[1:]

        features.append(row)

    return features

posts = pd.DataFrame(columns=['user_id', 'post_tags', 'direct'])

for user_id in tqdm(ego_users['user_id']):
    feat_names = pd.read_csv(os.path.join(TWITTER_DATA_DIR, f'{user_id}.featnames'), delimiter=' ', header=None, index_col=0).iloc[:, 0]
    ego_features = np.array(next(csv.reader(pd.read_csv(os.path.join(TWITTER_DATA_DIR, f'{user_id}.egofeat')), delimiter=' '))).astype(int)

    user_features_all = pd.DataFrame({'features': ego_features, 'names': feat_names})
    posts.loc[len(posts)] = [int(user_id), user_features_all[user_features_all['features'] == 1]['names'].values, 1]

    for row in csv.reader(open(os.path.join(TWITTER_DATA_DIR, f'{user_id}.feat'), encoding='latin1'), delimiter=' '):
        row_features = pd.DataFrame({'features': row[1:], 'names': feat_names})
        selected_row_features = list(row_features[row_features['features'] == '1']['names'])

        if len(selected_row_features):
            posts.loc[len(posts)] = [int(user_id), selected_row_features, 0]
    
posts.to_pickle(POSTS_DATA)

posts